In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
input_size = 4
hidden1_size = 3
hidden2_size = 2
ae_lr = 0.001
ft_lr = 0.001
num_epochs = 50000

In [ ]:
train = np.array(pd.read_csv('iris_training.csv'))
train_target = train[:,4]
train = train[:,:4]
test = np.array(pd.read_csv('iris_test.csv'))
test_target = test[:,4]
test = test[:,:4]

train_target = tf.one_hot(train_target, 3)
test_target = tf.one_hot(test_target, 3)

In [ ]:
## autoencoder
x = tf.placeholder(tf.float32, [None, input_size])
y = tf.placeholder(tf.float32, [None, 3])   

def autoencoder(x):
    # encoding
    w1 = tf.Variable(tf.random_normal([input_size, hidden1_size]))   
    b1 = tf.Variable(tf.random_normal([hidden1_size]))
    h1 = tf.nn.sigmoid(tf.matmul(x, w1) +b1)
    w2 = tf.Variable(tf.random_normal([hidden1_size, hidden2_size]))
    b2 = tf.Variable(tf.random_normal([hidden2_size]))
    h2 = tf.nn.sigmoid(tf.matmul(h1, w2) +b2)
    # decoding
    w3 = tf.Variable(tf.random_normal([hidden2_size, hidden1_size]))
    b3 = tf.Variable(tf.random_normal([hidden1_size]))
    h3 = tf.nn.sigmoid(tf.matmul(h2, w3) +b3)
    wo = tf.Variable(tf.random_normal([hidden1_size, input_size]))
    bo = tf.Variable(tf.random_normal([input_size]))
    x_reconstructed = tf.nn.sigmoid(tf.matmul(h3, wo) + bo)
    return x_reconstructed, h2 

In [ ]:
## softmax
def softmax_clf(x):
    w_softmax = tf.Variable(tf.zeros([hidden2_size, 3]))
    b_softmax = tf.Variable(tf.zeros([3]))
    y_pred = tf.nn.softmax(tf.matmul(x, w_softmax) + b_softmax)
    return y_pred

In [ ]:
x_pred, z = autoencoder(x)
y_pred = softmax_clf(z)
## pretraining
ae_loss = tf.reduce_mean(tf.pow(x - x_pred, 2))
ae_train_step = tf.train.AdamOptimizer(ae_lr).minimize(ae_loss)
## finetuning
finetuning_loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred)))
finetuning_train_step = tf.train.GradientDescentOptimizer(ft_lr).minimize(finetuning_loss)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(num_epochs):
        _, pt_loss = sess.run([ae_train_step, ae_loss], feed_dict={x:train})
        if epoch % 1000 == 0:
            print("pretraining -> epoch: {}, loss: {}".format(epoch, pt_loss))
    print("Pretaining finished")
    
    for epoch in range(num_epochs):
        _, ft_loss = sess.run([finetuning_train_step, finetuning_loss], feed_dict={x:train, y:sess.run(train_target)})
        if epoch % 1000 == 0:
            print("finetuning -> epoch: {}, loss: {}".format(epoch, ft_loss))
    print("fine-tuning finished")
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_pred,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("accuracy : {}".format(sess.run(accuracy, feed_dict={x: test, y: sess.run(test_target)})))

pretraining -> epoch: 0, loss: 12.433703422546387
pretraining -> epoch: 1000, loss: 10.313177108764648
pretraining -> epoch: 2000, loss: 10.106502532958984
pretraining -> epoch: 3000, loss: 10.062602043151855
pretraining -> epoch: 4000, loss: 10.04568862915039
pretraining -> epoch: 5000, loss: 10.037540435791016
pretraining -> epoch: 6000, loss: 10.033167839050293
pretraining -> epoch: 7000, loss: 10.030682563781738
pretraining -> epoch: 8000, loss: 10.029219627380371
pretraining -> epoch: 9000, loss: 10.028345108032227
pretraining -> epoch: 10000, loss: 10.027816772460938
pretraining -> epoch: 11000, loss: 10.027490615844727
pretraining -> epoch: 12000, loss: 10.027295112609863
pretraining -> epoch: 13000, loss: 10.027172088623047
pretraining -> epoch: 14000, loss: 10.027095794677734
pretraining -> epoch: 15000, loss: 10.02705192565918
pretraining -> epoch: 16000, loss: 10.027022361755371
pretraining -> epoch: 17000, loss: 10.027006149291992
pretraining -> epoch: 18000, loss: 10.02699